# A Structured Self-Attentive Sentence Embedding

https://arxiv.org/abs/1703.03130 논문에 대한 내용입니다.  

해당 논문은 sentence embedding을 위한 self-attention mechanism과 정규화를 위해 새로운 regularization term을 소개한다. 뿐만 아니라 추가적으로 visualizing 을 쉽게 할수록 설계 되어 있어 간단하게 설명 할 수 있도록 합니다.

### 1. Introduction

Word embedding 기법, 즉 개별 단어들에 대해 유의미한 distributed representation을 학습하는 기법들은 계속해서 많은 발전을 이뤄왔습니다.  
반면 아직 phrase나 sentence의 representation을 만드는 데는 word에 비해 아직은 부족한 상황입니다.  
보통 이와 같이 phrase나 sentence를 representation 하는 방법은 두가지로 나뉩니다.  
- 1. unsupervise 학습을 사용해 universal sentence representation을 만드는 방법입니다. (SkipThought vector, ParagraphVector, recursive auto-encoders, Sequential Denoising Autoencoder, FastSent, etc)
- 2. 특정 Task를 위해 특별하게 학습하는 방법입니다. 이러한 방법은 보통 supervised 학습하고, downstream application과 합쳐져서 사용됩니다. 그리고 몇몇 모델의 경우에는 일반적인 단어 임베딩을 사용하고 중간에 recurrent networks, recursive networks, convolutional networks 등을 사용함으로써 sentence represntation을 얻어 다양한 task에 적용됩니다.

__(downstream) vs (upstream)__ :
쉽게 말해 석유를 시추하는것을 downstream 이라하고 그걸 유통하는과정을 upstream이라고 하는데 ai쪽에선 downstream이 잘되야 upstream이 잘된다, 이정도로 알면 될거 같다,, 

Attention mechanism을 CNN혹은 LSTM네트워크 상단에 적용함으로써 추가적인 정보를 통해 sentence embedding을 추출하는 모델이 몇가지 Task에서 제안했습니다만 sentiment analysis같은 단일 문장이 입력으로 들어가는 경우에 추가적인 정보로 활용할 문장이 없기 때문에 attention mechanism을 적용할 수가 없었습니다.  

따라서 대부분의 경우에는 max or average pooling 기법을 적용하거나 RNN의 마지막 hidden vector를 선택해서 사용하는데, 해당 모델에서는 self attention 기법을 통해서 기존의 방법들을 대체합니다. self-attention의 경우에는 추가적인 입력값이 없는 하나의 문장에 대해서도 적용할 수 있고, 긴 문장에 대해서도 좋은 성능을 냅니다.

__max or average pooling__ :

### 2. Approach

##### 2.1 Model

Sentence embedding 모델은 크게 두개의 part로 구성되어 있습니다.  

- 1. 첫번째 part는 bidirectional LSTM을 사용하는 부분이고 다음은 self-attention을 적용하는 방법입니다. 
- 2. 두번째 part에서 나오는 값들을 사용해 LSTM의 hidden state값을 Weighted sum 하게 되고 이 값이 입력 문장에 대한 embedding vector로 사용합니다. 그리고 이값을 활용하여 각각의 task에 맞게 추가적인 networks 를 모델 상단에 적용시킬 수가 있습니다. 예를들면 sentence embedding vector에 multi-layer perceptron을 적용시켜서 sentiment analysis task 에 적용할 수 있습니다.  __EX)__ sentence embedding vector에 multilayer perceptron을 적용시켜 sentiment analysis task 에 적용

![](https://i.imgur.com/Ao2kIqr.png)

모델의 세부 과정에 대해 알아보겠습니다.  우선 아래와 같이 N개의 token을 가지는 입력 문장이 있다고 합니다. 아래와 같이 n개의 token을 가진 문장이 있다고 합시다. 입력 문장은 아래와 같이 각 단어들의 vector들이 모여서 matrix가 됩니다.
S = (w1, w2, w3, ..., wn)
여기서 wi는 i번째 단어의 d-dimensional vector이다. 입력 문장 S는 (n,d)형태가 됩니다. 해당 입력 문장에 bidirectional LSTM을 적용시켜 두개의 u-dimensonal hidden vector 값을 구합니다.  

h⟶ =LSTM⟶(wt,ht−1⟶)h←=LSTM⟵(wt,ht+1⟵)  

Bidrectional LSTM hidden state인 h⟶와 h⟵ 를 concatence한 결과인 ht를 사용합니다 전체길이 n에 대해 다음과 같이 n개의 hidden state값이 나오게 됩니다. 이 값들을 모아 하나의 matrix로 만들면 (n,2u)의 size를 갖게 됩니다.  
H = (h1, h2, ...., hn)  
가변 길이의 입력값에 대해서 동일한 크기의 embedding 값을 얻는 것을 목표로 하기 때문에 n개의 LSTM state를 적당한 linear combination을 통해 일정한 크기로 만들어 주야 한다. 여기서는 self-Attention mechanism 을 linear combination으로 사용합니다. Attention mechanism의 입력으로는 H를 사용하고, Weight로 사용되는 output __a__가 나옵니다.  
- a=softmax(ws2tanh(Ws1HT))  
여기서 Ws1은 (d_a,2u) 크기의 가중치 행렬이고, ws2는 (d_a) dimention의 가중치 벡터입니다. 최종 output인 a는 n dimension의 벡턱 나오게 됩니다. 해당 값은 각 token에 대해 얼마나 반영할지를 확률값으로 표현하게 됩니다. 이 값을 사용하여 H의 가중합을 구하게 됩니다.    
- m = sum(a*H)  

이 값은 한 문장에 대해서 하나의 semantic(의미) 정보를 담고있습니다. 하지만 일반적으로 문장의 경우에 여러개의 의미를 담는 경우가 많이 있습니다. 예를 들면 'and'로 연결되어 있는 문장의 경우 한문장이더라도 여러개의 의미를 담고 있습니다. 따라서 이러한 전체적인 의미를 담은 represent 하기 위해서 multiple __m__을 필요로 합니다. 따라서 multiple hope of attention을 사용하게 됩니다. 문장에서 r개의 각각 다른 부분의 의미를 추출하기 위해서 기존의 ws2를 (r,d_a)크기의 가중치 행렬로 확장시켜 다음과 같이 attention matrix를 구하게 됩니다.    
- A=softmax(Ws2tanh(Ws1HT))   

이후 최종 output은 위의 attention matrix A와 H를 행렬곱해서 얻게 됩니다. 